# 要約 
このJupyter Notebookは、Kaggleの「20の質問」ゲームコンペティションにおいて、AIエージェントを開発するための実装を行っています。具体的には、質問を投げかける「質問者」と、YES/NO形式で回答する「回答者」による対戦形式のゲームをシミュレーションしています。

### 問題の概要
ノートブックは、言語モデルを使用して「20の質問」ゲームをプレイし、特定のキーワード（最初は「日本」と設定される）を推測するエージェントを構築することを目的としています。エージェントはターンごとに質問を生成し、それに対する回答を受け取り、その情報に基づいて推測を行います。

### 使用している手法やライブラリ
1. **PyTorch**: モデルのトレーニングおよび推論に使用されており、GPUを利用して効率的に計算を行います。
2. **Transformers**: Hugging FaceのTransformersライブラリが使用され、様々な事前学習済みモデルを利用して質問や回答生成を行います。具体的には、`AutoTokenizer`および`AutoModelForCausalLM`を使用しています。
3. **BitsAndBytes**: モデルの量子化のための設定を行い、メモリ使用量を削減しつつ推論を最適化しています。
4. **Observationクラス**: ゲームの状態を管理するためのクラスで、質問、回答、推測を記録する属性を持っています。

### ノートブックの主な機能
- **モデルのロード**: 事前学習済みのモデルをGPUに最適化してロードします。
- **質問・回答の生成**: `generate_answer`関数を用いて、質問者が入力する質問に対する回答を生成します。
- **Robotクラス**: 質問者または回答者としての機能を持ち、各モードに応じた処理を行います。
- **エージェントの実装**: エージェントの主要なロジックを含む`agent`関数があり、観察結果に基づいて質問、回答、推測を行います。
- **ゲームのループ**: 最大20ラウンドのゲームを実装しており、各ターンで質問、回答、推測の結果を表示します。

ノートブックの最後では、運が良ければ指定されたキーワードを推測し、勝利を収めることを目指しています。このプロセスには、実行するたびに動的に選ばれる質問が含まれ、エージェントの戦略を強化するための試行が行われます。

---


# 用語概説 
以下に、対象のJupyter Notebookに関連する専門用語の解説を列挙します。初心者がつまずきやすいマイナーな用語やドメイン特有のキーワードに焦点を当てています。

1. **BitsAndBytes**:
   - メモリ効率のよい重みの保存と読み込みを可能にするライブラリです。通常の浮動小数点数よりも8ビットの整数を使用して、モデルのサイズを圧縮し、計算リソースを節約します。特にGPUメモリの制限がある場合に役立ちます。

2. **CUDAのメモリ効率的なSDP (Scalable Data Parallel)**:
   - 複数のGPUでのデータ並列処理を効率的に行うための手法ですが、メモリ使用効率を高めるために通常とは異なる方法で動作します。特に大規模モデルを扱う際に、リソースを節約しながらトレーニングできます。

3. **デバイスマッピング (device maps)**:
   - モデルの各レイヤーを特定のデバイス（通常はGPU）に割り当てるための設定です。レイヤーごとに異なるGPUに分散配置することで、メモリ負荷を分散し、効率的な計算を実現します。

4. **量子化 (quantization)**:
   - モデルの重みや計算を低精度（例えば8ビット）で表現するプロセスです。これによりモデルのサイズが小さくなり、速度が向上します。量子化は特にリソース制約が厳しい環境やデバイスにおいて重要です。

5. **勾配チェックポイント (gradient checkpointing)**:
   - トレーニング中にメモリを節約し、計算グラフの一部を保存しながら逆伝播の際に必要な部分だけを再計算する手法です。これにより、より大きなモデルを小さいGPUメモリでトレーニング可能になります。

6. **ターニングタイプ (turnType)**:
   - ゲーム内でエージェントが行うアクションの種類を示す指標です。具体的には、質問する（ask）、回答する（answer）、推測する（guess）の3種類があります。この概念はゲームの進行において重要です。

7. **エージェント (agent)**:
   - 環境内での行動をコントロールするプログラムやAIのことです。この競技では、質問者と回答者の役割を持つエージェントが、それぞれのタスクを実行します。

8. **観察クラス (Observation class)**:
   - ゲームの進行状況や状態を保持するために使用されるクラスです。このクラスには現在のステップ、質問&回答履歴、ターゲットキーワードやそのカテゴリなどが含まれます。

9. **キーワードデータフレーム (keywords DataFrame)**:
   - ゲーム内で使用されるキーワードやそのカテゴリ、代替キーワードの情報を整理して保持するためのデータ構造です。これを使用することで、ゲームの進行に必要な情報を効率的に管理します。

これらの用語を理解することで、ノートブック内のコードやその意図をより深く理解できるようになるでしょう。

---


In [ ]:
%%bash
# サブミッション用のディレクトリを作成します。-pオプションは、親ディレクトリが存在しない場合にそれも作成します。
mkdir -p /kaggle/working/submission

# bitsandbytesとaccelerateパッケージをインストールします。
pip install bitsandbytes accelerate

# transformersライブラリを最新のバージョンにアップグレードします。
pip install -U transformers

In [ ]:
%%writefile -a submission/main.py

import os
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig

# CUDAのメモリ効率的なSDPを無効にします。
torch.backends.cuda.enable_mem_efficient_sdp(False)

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
# Kaggleエージェントのパスが存在する場合、モデルIDを設定します。
if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, "model")
#else:
# とりあえず別のモデルパスを指定します。
model_id = "/kaggle/input/gemma/transformers/7b-it/3"

# 各レイヤーに対するデバイスマッピングを定義します。
device_maps = [('model.layers.0', 0),
 ('model.layers.1', 0),
 ('model.layers.2', 0),
 ('model.layers.3', 0),
 ('model.layers.4', 0),
 ('model.layers.5', 0),
 ('model.layers.6', 0),
 ('model.layers.7', 0),
 ('model.layers.8', 0),
 ('model.layers.9', 0),
 ('model.layers.10', 0),
 ('model.layers.11', 0),
 ('model.layers.12', 0),
 ('model.layers.13', 0),
 ('model.layers.14', 0),
 ('model.layers.15', 0),
 ('model.layers.16', 0),
 ('model.layers.17', 0),
 ('model.layers.18', 0),
 ('model.layers.19', 1),
 ('model.layers.20', 1),
 ('model.layers.21', 1),
 ('model.layers.22', 1),
 ('model.layers.23', 1),
 ('model.layers.24', 1),
 ('model.layers.25', 1),
 ('model.layers.26', 1),
 ('model.layers.27', 1),
 ('model.layers.28', 1),
 ('model.layers.29', 1),
 ('model.layers.30', 1),
 ('model.layers.31', 1),
 ('model.layers.32', 1),
 ('model.layers.33', 1),
 ('model.layers.34', 1),
 ('model.layers.35', 1),
 ('model.layers.36', 1),
 ('model.layers.37', 1),
 ('model.layers.38', 1),
 ('model.layers.39', 1),
 ('model.layers.40', 1),
 ('model.layers.41', 1),
 ('model.embed_tokens', 1),
 ('model.layers', 1)]

# 量子化設定を8ビットで読み込むように設定します。
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
# トークナイザーを初期化します。
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3") 
id_eot = tokenizer.convert_tokens_to_ids(["<eos>"])[0]

# デバイスマップを設定して、モデルを読み込みます。
device = {layer: gpu_mem for (layer, gpu_mem) in device_maps}
config = AutoConfig.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3")
# 勾配チェックポイントを有効にします。
config.gradient_checkpointing = True
# モデルを指定されたパスから読み込みます。
model = AutoModelForCausalLM.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3", torch_dtype="auto", quantization_config=quantization_config,
                                             device_map="auto", trust_remote_code=True, config=config)

# 回答生成関数
def generate_answer(template):
    # テンプレートからトークンIDを取得します。
    input_ids = tokenizer(template, return_tensors="pt").to("cuda")
    # モデルから出力トークンIDを生成します。
    output_ids = model.generate(**input_ids, max_new_tokens=15).squeeze()
    start_gen = input_ids.input_ids.shape[1]
    output_ids = output_ids[start_gen:]
    # 出力が終了トークンを含むか確認し、必要に応じてデコードします。
    if id_eot in output_ids:
        stop = output_ids.tolist().index(id_eot)
        output = tokenizer.decode(output_ids[:stop])
    else:
        output = tokenizer.decode(output_ids)
    # 不要な改行や末尾のトークンを削除します。
    output = re.sub('\n', '', output)
    output = re.sub(' <end_of_turn>', '', output)
    output = re.sub('<end_of_turn>', '', output)
    return output

# Robotクラスの定義
class Robot:
    def __init__(self):
        pass
    
    def on(self, mode, obs):
        # モードに応じて処理を分岐します。
        assert mode in ["asking", "guessing", "answering"], "mode can only take one of these values: asking, answering, guessing"
        if mode == "asking":
            output = self.asker(obs)
        if mode == "answering":
            output = self.answerer(obs)
            # 出力を「yes」または「no」に規定します。
            if "yes" in output.lower() or "Yes" in output.lower():
                output = "yes"
            elif "no" in output.lower() or "No" in output.lower():
                output = "no"
            else:
                output = "yes"
        if mode == "guessing":
            output = self.asker(obs)
        return output

    def asker(self, obs):
        # 質問する際のシステムプロンプト
        sys_prompt = """
        あなたは「20の質問」ゲームをプレイするために設計されたAIアシスタントです。 
        このゲームでは、回答者がキーワードを考え、質問者がイエス/ノーで質問をしてそれに応えます。
        キーワードは特定の国です。
        """
        # 質問を投げかける場合の処理
        if obs.turnType == "ask":
            ask_prompt = sys_prompt + """
            では20の質問を始めましょう。あなたは質問者の役割を果たしています。
            YesまたはNoで答えられる質問をしてください。
            例として、キーワードがモロッコの場合、次のように流れます:
            例:
            <あなた: 国ですか？
            ユーザー: はい
            あなた: ヨーロッパにありますか？
            ユーザー: いいえ
            あなた: アフリカにありますか？
            ユーザー: はい
            あなた: その国に住む人は大半が肌の色が濃いですか？
            ユーザー: いいえ
            あなた: 「m」で始まる国の名前ですか？
            ユーザー: はい
            あなた: モロッコですか？
            ユーザー: はい。>
            ユーザーが選んだ言葉を考えて、最初の質問をしてください。
            できるだけ短く、冗長にならないように、1つの質問だけをしてください。余計な言葉は不要です！
            """
        
            chat_template = f"""<start_of_turn>system\n{ask_prompt}<end_of_turn>\n"""
            chat_template += "<start_of_turn>model\n"

            if len(obs.questions) >= 1:
                # 質問と回答の履歴を追加します。
                for q, a in zip(obs.questions, obs.answers):
                    chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                    chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
        
        # 残りのラウンドの推測処理
        elif obs.turnType == "guess":
            conv = ""
            for q, a in zip(obs.questions, obs.answers):
                conv += f"""Question: {q}\nAnswer: {a}\n"""
            guess_prompt = sys_prompt + f"""
            現在のゲームの状態は以下の通りです:\n{conv}
            この会話に基づいて、言葉を推測できますか？できるだけ簡潔に、言葉だけを提供してください。
            """
            chat_template = f"""<start_of_turn>system\n{guess_prompt}<end_of_turn>\n"""
            chat_template += "<start_of_turn>model\n"

        output = generate_answer(chat_template)        
        return output
    
    # 回答者の処理
    def answerer(self, obs):
        sys_prompt = f"""
        あなたは「20の質問」ゲームをプレイするために設計されたAIアシスタントです。 
        このゲームでは、回答者がキーワードを考え、質問者がイエス/ノーで質問をしてそれに応えます。
        キーワードは特定の場所または物です。\n
        ユーザーの質問を理解し、推測されるキーワードを把握してください。
        現在、ユーザーが推測すべき言葉は: "{obs.keyword}" で、カテゴリは "{obs.category}" です。
        例として、キーワードがモロッコかつカテゴリが「場所」の場合、次のように流れます:
        例:
        <ユーザー: それは場所ですか？
        あなた: はい
        ユーザー: ヨーロッパにありますか？
        あなた: いいえ
        ユーザー: アフリカにありますか？
        あなた: はい
        ユーザー: その土地に住む人々は肌の色が暗いですか？
        あなた: いいえ
        ユーザー: 「m」で始まる国名ですか？
        あなた: はい
        ユーザー: モロッコですか？
        あなた: はい。>
        """
        chat_template = f"""<start_of_turn>system\n{sys_prompt}<end_of_turn>\n"""
        chat_template += "<start_of_turn>user\n"
        chat_template += f"{obs.questions[0]}"
        chat_template += "<start_of_turn>model\n"
        # 以前の質問と回答をテンプレートに追加します。
        if len(obs.answers) >= 1:
            for q, a in zip(obs.questions[1:], obs.answers):
                chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
        output = generate_answer(chat_template)
        return output

# Robotクラスのインスタンスを生成します。
robot = Robot()

# エージェントのメイン関数
def agent(obs, cfg):
    # 各ターンタイプに応じてロボットの処理を呼び出します。
    if obs.turnType == "ask":
        response = robot.on(mode="asking", obs=obs)
        
    elif obs.turnType == "guess":
        response = robot.on(mode="guessing", obs=obs)
        
    elif obs.turnType == "answer":
        response = robot.on(mode="answering", obs=obs)
        
    # レスポンスが無効な場合、デフォルトとして「yes」を返します。
    if response is None or len(response) <= 1:
        response = "yes"
        
    return response

In [ ]:
def simple_agent1(obs, cfg):
    # エージェントが推測者で、ターンタイプが「ask」の場合
    if obs.turnType == "ask":
        # 質問のリストを定義します。
        response_list = [
            'それはアフリカにありますか？',
            'それはアメリカにありますか？',
            'それはアジアにありますか？',
            'それはオセアニアにありますか？',
            'それは東ヨーロッパにありますか？',
            'それは北ヨーロッパにありますか？',
            'それは南ヨーロッパにありますか？',
            'それは西ヨーロッパにありますか？',
            'それは日本ですか？'
        ]
        # 現在の質問の数に基づいて、質問を選ぶ（コメントアウトされたが、長さに基づくロジックのサンプルあり）。
#         response = response_list[len(obs.questions)]
        # ランダムに質問を選びます。
        response = random.choice(response_list)
    elif obs.turnType == "guess":
        # 推測する際のデフォルトの応答
        response = "duck"  # 推測する単語が「アヒル」です。
    elif obs.turnType == "answer":
        # イエスまたはノーをランダムに選びます。
        response = random.choices(["yes", "no"])[0]
    return response

In [ ]:
#!apt install pigz pv > /dev/null
# pigzとpvをインストールします。
# pigzは圧縮ツールで、pvはデータの進捗状況を表示するためのコマンドラインツールです。
# これらのインストール出力を非表示にします（> /dev/null）。

In [ ]:
#!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/transformers/2b-it/2
# tarコマンドを使用して、指定したディレクトリを圧縮アーカイブにします。
# pigzを使って圧縮し、pvを使って進捗状況を表示します。
# submission.tar.gzという名前でアーカイブが作成されます。
# -Cオプションは、アーカイブに追加する前に現在の作業ディレクトリを指定します。
# 最初の -C は /kaggle/working/submission ディレクトリに移動し、その内容をアーカイブします。
# 次の -C は /kaggle/input/gemma/transformers/2b-it/2 に移動し、このディレクトリもアーカイブに追加します。

In [ ]:
#%%time
# 実行時間を計測します。

#import random  # randomモジュールをインポートします。
#from kaggle_environments import make  # Kaggle環境を作成するためのmake関数をインポートします。
# agent変数にエージェントのパスを指定します。
#agent = "/kaggle/working/submission/main.py"
# 環境を作成します。ここでは「llm_20_questions」ゲームをデバッグモードで生成します。
#env = make("llm_20_questions", debug=True)
# 環境でエージェントを実行します。最初のエージェントは自作エージェント、その他はsimple_agent1です。
#game_output = env.run(agents=[agent, simple_agent1, simple_agent1, simple_agent1])

In [ ]:
#env.render(mode="ipython", width=600, height=500)
# 環境の描画を行います。ここではIPythonモードで表示し、幅600ピクセル、高さ500ピクセルに設定します。

In [ ]:
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py
# 指定されたURLからkeywords.pyファイルをダウンロードし、keywords_local.pyという名前で保存します。
# wgetコマンドを使用してインターネットからファイルを取得します。

In [ ]:
import json
import pandas as pd
from submission.main import agent  # 作成したエージェントをインポートします。
from keywords_local import KEYWORDS_JSON  # ダウンロードしたキーワードのJSONデータをインポートします。

# 観察用クラスの定義
class Observation:
    def __init__(self):
        self.step = 0  # 現在のステップ数を初期化します。
        self.role = "guesser"  # 役割を「推測者」に設定します。
        self.turnType = "ask"  # 現在のターンタイプを「質問」に設定します。
        self.keyword = "Japan"  # キーワードを初期化します。「日本」をデフォルトとします。
        self.category = "country"  # カテゴリーを「国」に設定します。
        self.questions = []  # 質問のリストを初期化します。
        self.answers = []  # 回答のリストを初期化します。
        self.guesses = []  # 推測のリストを初期化します。
        
# JSONデータからキーワードのデータフレームを作成する関数
def create_keyword_df(KEYWORDS_JSON):
    json_data = json.loads(KEYWORDS_JSON)  # JSONをPythonの辞書に変換します。

    keyword_list = []  # キーワードを格納するリスト
    category_list = []  # カテゴリーを格納するリスト
    alts_list = []  # 代替キーワードを格納するリスト

    # JSONデータをループし、各キーワードを抽出します。
    for i in range(len(json_data)):
        for j in range(len(json_data[i]['words'])):
            keyword = json_data[i]['words'][j]['keyword']  # キーワードを取得します。
            keyword_list.append(keyword)  # キーワードリストに追加します。
            category_list.append(json_data[i]['category'])  # カテゴリーリストに追加します。
            alts_list.append(json_data[i]['words'][j]['alts'])  # 代替キーワードリストに追加します。

    # データフレームを作成し、各リストをカラムに設定します。
    data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])
    data_pd['keyword'] = keyword_list
    data_pd['category'] = category_list
    data_pd['alts'] = alts_list
    
    return data_pd  # 作成したデータフレームを返します。
    
# キーワードデータフレームを作成します。
keywords_df = create_keyword_df(KEYWORDS_JSON)

In [ ]:
obs = Observation()  # 観察オブジェクトを作成します。
cfg = "_"  # 設定用の変数（未使用）

# キーワードデータフレームからランダムにサンプルを取得します。
sample_df = keywords_df.sample()
obs.keyword = sample_df["keyword"].values[0]  # キーワードを設定します。
obs.category = sample_df["category"].values[0]  # カテゴリーを設定します。
alts_list = sample_df["alts"].values[0]  # 代替キーワードリストを取得します。
alts_list.append(obs.keyword)  # 元のキーワードを代替キーワードリストに追加します。

print(f"keyword: {obs.keyword}")  # 現在のキーワードを表示します。

# 最大20ラウンドのゲームを開始します。
for round in range(20):
    obs.step = round + 1  # ステップ数を更新します。
    
    obs.role = "guesser"  # 現在の役割を「推測者」に設定します。
    obs.turnType = "ask"  # 現在のターンタイプを「質問」に設定します。
    question = agent(obs, cfg)  # エージェントに質問を投げます。
    obs.questions.append(question)  # 質問を履歴に追加します。
    
    obs.role = "answerer"  # 役割を「回答者」に設定します。
    obs.turnType = "answer"  # ターンタイプを「回答」に設定します。
    answer = agent(obs, cfg)  # エージェントに回答を求めます。
    obs.answers.append(answer)  # 回答を履歴に追加します。
    
    obs.role = "guesser"  # 役割を再度「推測者」に設定します。
    obs.turnType = "guess"  # ターンタイプを「推測」に設定します。
    guess = agent(obs, cfg)  # エージェントに推測を求めます。
    obs.guesses.append(guess)  # 推測を履歴に追加します。
    
    # 現在のラウンド、質問、回答、推測を表示します。
    print(f"round: {round + 1}")
    print(f"question: {question}")
    print(f"answer: {answer}")
    print(f"guess: {guess}")
    
    # 推測が代替リストに含まれている場合、勝利を表示してループを終了します。
    if guess in alts_list:
        print("Win!!")  # 勝利のメッセージを表示します。
        break  # ゲームを終了します。